# More velocity graph shit

Okay, picking back up on [yesterday's work...](2025-12-11.ipynb)

I have some more thoughts.

## You don't need that many subdivisions

There is only so much detail required in meter. There is a point where further subdivisions of a meter cease to be meaningful. 

Like, if two drums are only like, 10ms apart, or some other sufficiently small time delta, then they stop being perceived as distinct events and blend into a single texture.

## An idea for building velocity patterns

So, I think you start by specifying the base pattern with an onset function.

For example, `OnsetFunc([1 1/2, 2 1/2], offset=1/2)`.

Then, you can subdivide this by calling `.subdivide` or something with another onset function as the argument, returning a new meter.

## Time resolution matching

One open question: is it meaningful to impose a velocity graph that expresses one meter over a clip that expresses another?

For example, if the meter of the clip in question follows a straight 16th note 4/4 type rhythm, is it meaningful to impose a velocity graph over it that follows a 9/8 triplet type rhythm?

My guess is that this is probably dependent on the interpolation being used. Maybe less likely with sample and hold, and more likely with linear or exponential etc interpolation.

I want to investigate this question. 

And maybe speaking in terms of "meter" isn't the best. Because meter is also about the pattern of stressed and unstressed notes. 

I think it's more specific to say "resolution". By resolution, I mean the resolution of the time grid that onsets fall on. Maybe some onsets fall on multiples of 1, and some fall on multiples of 1/2, and some fall on multiples of 1/3. That means there's a resolution of 1/6, because `1/3 * 1/2 = 1/6`. I think the way to calculate the resolution is to take inventory of all prime subdivisions of the beat that are present, and multiply the smallest of these together. In this case, the prime subdivisions present are 2 and 3. The smallest 2-subdivision is 1/2, and the smallest 3-subdivision is 1/3, so you multiply them to get 1/6. That's my guess.

Anyways, I think it's most meaningful to match clips with velocity graphs that have the same time resolution. 

So if the time resolution of a clip is 1/6 a beat, then it's probably best to impose a velocity graph that also has a time resolution of 1/6.

## Returning to pattern building

Okay, so my question - is it as simple as stacking layers of onset patterns? Perhaps so.

Maybe you just say, "Here's our first onset pattern, and it has a velocity of 1."

"Let's subdivide it with this second onset pattern, and give it a velocity of 2/3."

"Let's subdivide that with this third onset pattern, and give it a velocity of 1/3."

I remember thinking before about how you can subdivide a region in different orders but arrive at the same resolution.

---

The patterns can be specified in terms of deltas.

[3/2, 1]. That's the first level.

This results in an onset pattern [3/2, 5/2].

Maybe it's just a matter of partitioning each delta into smaller deltas.

It's a partitioning problem!

---

Alright, I worked this out on paper, and here's what I got.

## The subdivision algorithm

So basically, you might as well start out with a specific resolution in mind. Let's say the resolution is 1/4 a beat - sixteenth notes.

Then, you decide on a region of time to partition. I want to do something with a time signature of 5/8, so let's partition 10/4 beats.

Now you simply start partitioning this region iteratively. We decide on our first partition - this also includes the possibility of not partitioning at all, which would basically just result in the first subdivision consisting of a single onset. In our case, we partition 10 into 6 and 4. Then we assign this first level a velocity. We'll choose 1.

Note about velocities: the basic idea is that velocities diminish with each finer level, but technically you don't have to follow this if you really don't wanna?

Anyways, after each subdivision, we're left with two choices. We can either continue subdividing like we just did, or we can do one final subdivision that wraps up the process by subdividing everything into our smallest unit - the resolution unit, or w/e you wanna call it - in this case, 1/4. This would be the final subdivision level, and it would also be assigned a velocity.

Otherwise, if we just wanna repeat the subdivision process, then we take our previous partition and we partition each of its parts again.

So we have 6/4 and 4/4. We can partition 6 into 3 and 3, giving us 3/4 and 3/4. And 4/4 can be partitions into 2/4 and 2/4. So our next partition level is [3/4, 3/4, 2/4, 2/4], and we assign it a velocity of 2/3.

Note about partitions: A number is its own partition, so if we have any parts that we just don't want to partition on any given subdivision level, we can just do that. For example, if we wanted to further subdivide 3/4 and 3/4 into parts, but leave 2/4 and 2/4 alone, then that can be how it is on one level. 

### Side thought

What if the region of time has a different resolution than the one we declare?

Like, what if we want the resolution to be 1/4, but the region of time is 1 1/7?

You could take the remainder of this region, when divided by the resolution, and treat it as a resolution unit.

### How do we implement all this?

Not sure. Let me try stuff.

In [2]:
from harmonica.utility._mixed import Mixed

resolution = Mixed("1/4")
region = Mixed("5/2")

We need to get the resolution 10/4 from our region - 5/2 - and our resolution - 1/4. How do we get 10 from these two numbers? I guess we just divide 5/2 by 1/4.

In [3]:
first_part = region // resolution

So we look at the partitions of this.

In [ ]:
from harmonica.utility._utility import int_partitions


parts = [
    part
    for part in int_partitions(first_part)
    if len(part) < 4 and not any([n == 1 for n in part])
]

for part in parts:
    print(part)

(10,)
(2, 8)
(2, 2, 6)
(2, 3, 5)
(2, 4, 4)
(3, 7)
(3, 3, 4)
(4, 6)
(5, 5)


Alright, let's just enter it manually, yeah?

Like, we just enter our pattern as:

```
dur = 10
res = 1/4
subdivisions = (
    (6, 4),
    (3, 3, 2, 2),
    (1, 1, 1, 1, 1, 1, 1, 1, 1, 1),
)
velocities = (1, 2/3, 1/3)
```

How do we actually evaluate, though? Like, say in this example, I enter a time of 14 3/4. What velocity should it return?

Let's assume an offset value of 0 to simplify things.

Well, we take the remainder of 14 3/4 and our duration in beats, which is 2 1/2. What is that? 2 1/4.

I guess at this point we just iterate through our subdivisions.

We have to transform 

```
(
    (6, 4),
    (3, 3, 2, 2),
    (1, 1, 1, 1, 1, 1, 1, 1, 1, 1),
)
```

into 

```
(
    (0, 6, 10),
    (0, 3, 6, 8, 10),
    (0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10)
)
```

by prepending 0 to each and accumulating.

then into 

```
(
    (0, 6),
    (0, 3, 6, 8),
    (0, 1, 2, 3, 4, 5, 6, 7, 8, 9)
)
```

by removing the tail from each.

then into

```
(
    (0, 6),
    (3, 8),
    (1, 2, 4, 5, 7, 9)
)
```

by removing from each any numbers that appeared in any previous.

In [32]:
from dataclasses import dataclass
from itertools import accumulate

from harmonica.utility._mixed import Mixed


@dataclass
class VelocityFunc:
    duration: int
    resolution: Mixed
    subdivisions: list[list[int]]
    velocities: list[Mixed]
    offset: Mixed

    def __post_init__(self):
        assert (
            self.duration % self.resolution == 0
        ), "Duration must be a multiple of resolution."

        assert (
            self.offset % self.resolution == 0
        ), "Offset must be a multiple of resolution."

        assert all(
            [sum(parts) == self.duration for parts in self.subdivisions]
        ), "Sum of parts in each subdivision level must be equal to duration."

        assert all(
            [0 <= velocity <= 1 for velocity in self.velocities]
        ), "Velocities must be between 0 and 1."

    def evaluate(self, t: Mixed) -> Mixed:
        """Take an onset time t and return a velocity value.
        Naive algorithm that assumes t is a multiple of resolution."""

        assert (
            t % self.resolution == 0
        ), "Evaluated time t must be a multiple of resolution."

        remainder = (t + self.offset) % self.dur_in_beats

        for i, subdiv_level in enumerate(self._subdiv_onsets()):
            for onset_numerator in subdiv_level:
                if (onset_numerator * self.resolution) == remainder:
                    return self.velocities[i]

        return Mixed()

    def _subdiv_onsets(self) -> list[list[int]]:
        """Transform partitions into onsets that can be compared against while evaluating."""

        subdivs = [[0] + list(accumulate(subdiv[:-1])) for subdiv in self.subdivisions]

        result = [subdivs[0]]

        for i, subdiv in enumerate(subdivs[1:]):
            result.append([onset for onset in subdiv if onset not in subdivs[i]])

        return result

    @property
    def dur_in_beats(self) -> Mixed:
        return Mixed(self.duration) * self.resolution

Okay, I think I got it working! Let me make a drum beat and apply it.

In [45]:
from harmonica.time._event._clip import Clip
from harmonica.time._onsetfunc import OnsetFunc
from harmonica.utility._gm import GMDrum

vels = [1, "2/3", "1/3"]

vfunc = VelocityFunc(
    duration=10,
    resolution=Mixed("1/4"),
    subdivisions=[[6, 4], [3, 3, 2, 2], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],
    velocities=[Mixed(x) for x in vels],
    offset=Mixed("0"),
)

rhythm = OnsetFunc([Mixed("1/4")]).to_clip(duration=Mixed(64), drum=GMDrum.Cabasa)

for drum in rhythm.get_drum_events():
    drum.velocity = vfunc.evaluate(drum.onset)

vfunc2 = VelocityFunc(
    duration=9,
    resolution=Mixed("1/4"),
    subdivisions=[[5, 4], [3, 2, 2, 2], [1, 1, 1, 1, 1, 1, 1, 1, 1]],
    velocities=[Mixed(x) for x in vels],
    offset=Mixed("0"),
)

rhythm2 = OnsetFunc([Mixed("1/4")]).to_clip(duration=Mixed(64), drum=GMDrum.HiWoodBlock)

for drum in rhythm2.get_drum_events():
    drum.velocity = vfunc2.evaluate(drum.onset)

Clip([rhythm, rhythm2]).preview(124)

## SUCCESS!